In [1]:
import os
import numpy as np
os.environ['CFLAGS'] = '-fopenmp'
os.environ['LDFLAGS'] = '-fopenmp'

os.environ["C_INCLUDE_PATH"] = np.get_include()

import pyximport
pyximport.install()

from collections import defaultdict
from sklearn.metrics import roc_auc_score
from copy import deepcopy
from rocauc_pairwise.sigmoid_pairwise_cpu import sigmoid_pairwise_loss
from rocauc_pairwise.sigmoid_pairwise_auc_cpu import sigmoid_pairwise_loss_auc_cpu
from rocauc_pairwise.sigmoid_pairwise_auc_cpu import sigmoid_pairwise_loss_auc_exact_cpu

In [2]:
N = 10_000
y_true = np.random.randint(0, 2, N)
y_pred = np.random.randn(N)
exp_pred = np.exp(y_pred)

In [3]:
def get_pairs(y_true, y_pred):
    pairs = []
    for i in range(len(y_true)):
        for j in range(i + 1):
            pairs.append([(i, j), (y_true[i], y_true[j]), (y_pred[i], y_pred[j])])
    return pairs

In [4]:
def delta_auc_score(y_true, y_pred, i, j):
    auc_1 = roc_auc_score(y_true, y_pred)
    y_pred_ = deepcopy(y_pred)
    y_pred_[i], y_pred_[j] = y_pred_[j], y_pred_[i]
    auc_2 = roc_auc_score(y_true, y_pred_)
    return auc_1 - auc_2

In [5]:
def grad(pairs):
    grad_ = defaultdict(lambda : 0)
    hess_ = defaultdict(lambda : 0)
    loss = 0
    eps = 1e-20

    for p in pairs:
        (i, j), (y_true_i, y_true_j), (y_pred_i, y_pred_j) = p
        if(y_true_i > y_true_j):
            P_hat = 1.0
        elif y_true_j == y_true_i:
            P_hat = 0.5
        elif y_true_i < y_true_j:
            P_hat = 0.0
        
        deltaauc_ij = delta_auc_score(y_true, y_pred, i, j)

        grad_i = ((P_hat - 1.)*np.exp(y_pred_i) + P_hat*np.exp(y_pred_j)) / (np.exp(y_pred_i) + np.exp(y_pred_j))
        grad_j = -grad_i

        hess_i = -(np.exp(y_pred_i + y_pred_j)) / (np.exp(y_pred_i) + np.exp(y_pred_j))**2
        hess_j = hess_i

        grad_[i] += grad_i * np.abs(deltaauc_ij)
        grad_[j] += grad_j * np.abs(deltaauc_ij)
        
        hess_[i] += hess_i * np.abs(deltaauc_ij)
        hess_[j] += hess_j * np.abs(deltaauc_ij)

        exp_pred_i = np.exp(y_pred_i)
        exp_pred_j = np.exp(y_pred_j)
        P = 1.0 / (1.0 + (exp_pred_j / exp_pred_i))
        loss += np.abs(deltaauc_ij)*(P_hat*np.log(P + eps) + (1.0 - P_hat)*np.log(1.0 - P - eps))
    return grad_.values(), hess_.values(), loss

In [ ]:
y_true = np.array([1, 0, 0, 1, 0, 0, 1, 1])
y_pred = np.array([0.3, 0.1, 0.5, 0.5, 0.3, 0.1, 0.5, 0.24])

In [14]:
pairs = get_pairs(y_true, y_pred)

grad(pairs)

In [6]:

sigmoid_pairwise_loss_auc_cpu(y_true, exp_pred, 12)

-3445.547255752764

In [7]:
sigmoid_pairwise_loss_auc_exact_cpu(y_true, exp_pred, 12)

-3445.5472557527637